# Curating Input for Glyph Classification
#### Written by Kara Lynn Bressler for work w/ MIT Media Lab's Poetic Justice Group 2022
#### IN PROGRESS, TODO: Comment and link to other related projects

## Importing Libraries

In [2]:
import cairosvg
import cv2
import gspread
import math
import numpy as np
import os
import time

from IPython.display import SVG, display
from PIL import Image, ImageEnhance

from matplotlib import pyplot as plt
from reportlab.graphics import renderPDF, renderPM
from svglib.svglib import svg2rlg

# Mute tensorflow debugging information console
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

## Defining Constants

In [3]:
BLACK = [  0,   0,   0]
WHITE = [255, 255, 255]

### Now let's scrape this Google Doc (private permissions): 
##### https://docs.google.com/spreadsheets/d/1aInsuVSFt22PKEZRKgYTRy97Goy3f5W5BNyg0OL0Xuo/edit#gid=1113150339

### More information about setting up and using gspread to scrape Google Docs can be found here:
##### https://www.youtube.com/watch?v=bu5wXjz2KvU

In [5]:
def show_svg(str):
    display(SVG(url=str))

## Testing Out gspread

In [6]:
# Service Account
sa = gspread.service_account(filename="gspread/service_account.json")

# Sheet
sh = sa.open("RTR: Fortune 500 Logos")

# Worksheet
wks = sh.worksheet("Parent Companies")

print(wks.acell('G2').value)

#print('Rows: ', wks.row_count)
#print('Cols: ', wks.col_count)
#print(wks.cell(3,4))
#print(wks.get('A7:E9'))
#print(wks.get_all_records())
#print(wks.get_all_values)
#print.update('A3', 'Anthony')

https://upload.wikimedia.org/wikipedia/commons/c/ca/Walmart_logo.svg


In [22]:
NAME_COL = 'C'  # ie. "Walmart"
SVG_COL  = 'G'  # ie. "https://upload.wikimedia.org/wikipedia/commons/c/ca/Walmart_logo.svg"

row = 153

while row < 502:
    row              += 1
    company_cell_name = NAME_COL + str(row)
    svg_cell_name     = SVG_COL + str(row)
    
    company_name      = str(wks.acell(company_cell_name).value).lower().replace(" ", "_")
    svg_path          = str(wks.acell(svg_cell_name).value)
    png_path          = "full_png_height_1400/" + company_name + "_" + str(row) + ".png"
    
    # print(company_name)
    # print(svg_path)
    
    # Show the SVG
    # show_svg(svg_path)
    
    height_28 = 28
    height_1400 = 1400
    background_color = 'white' # white
    
    try: 
        cairosvg.svg2png(url=svg_path, 
                         write_to=png_path, 
                         background_color=background_color,
                         output_height=height_1400)
    except Exception as e: 
        print("Error: " + str(e))
        print("Could not scrape the photo for " + company_name)
        print()
    
    # Google Sheets API has a scraping limit per minute
    # Limit scraping to 10 calls per minute
    time.sleep(6)

Error: HTTP Error 404: Not Found
Could not scrape the photo for dupont

Error: EntitiesForbidden(name='ns_svg', system_id=None, public_id=None)
Could not scrape the photo for union_pacific

Error: not well-formed (invalid token): line 1, column 8
Could not scrape the photo for ecolab

Error: EntitiesForbidden(name='ns_extend', system_id=None, public_id=None)
Could not scrape the photo for fidelity_national_information_services

Error: HTTP Error 404: Not Found
Could not scrape the photo for ameriprise_financial

Error: <urlopen error [Errno 2] No such file or directory: '/Users/karabressler/Desktop/Topography Typography /Real Talk Radio/None'>
Could not scrape the photo for hollyfrontier

Error: HTTP Error 404: Not Found
Could not scrape the photo for lam_research

Error: HTTP Error 404: Not Found
Could not scrape the photo for molson_coors_beverage

Error: <urlopen error [Errno 2] No such file or directory: '/Users/karabressler/Desktop/Topography Typography /Real Talk Radio/None'>
Cou

Error: <urlopen error [Errno 2] No such file or directory: '/Users/karabressler/Desktop/Topography Typography /Real Talk Radio/None'>
Could not scrape the photo for a-mark_precious_metals

Error: HTTP Error 404: Not Found
Could not scrape the photo for r.r._donnelley_&_sons



APIError: {'code': 400, 'message': "Range ('Parent Companies'!C502) exceeds grid limits. Max rows: 501, max columns: 13", 'status': 'INVALID_ARGUMENT'}

In [26]:
row = 1

# assign directory
read_dir = "full_png_height_1400"
write_dir = "full_png_height_1400_bw"
 
# iterate over files in that directory
for filename in os.listdir(read_dir):
    f = os.path.join(read_dir, filename)
    # checking if it is a file
    if os.path.isfile(f):
        img_grey = cv2.imread(f, cv2.IMREAD_GRAYSCALE)

        # define a threshold, 128 is the middle of black and white in grey scale
        threshold = 254

        # threshold the image
        img_binary = cv2.threshold(img_grey, threshold, 255, cv2.THRESH_BINARY)[1]

        #save image
        img_save_name = write_dir + "/" + filename
        cv2.imwrite(img_save_name, img_binary) 

In [12]:
#https://www.geeksforgeeks.org/how-to-detect-shapes-in-images-in-python-using-opencv/

# Reading binary, b&w image
img = cv2.imread('full_png_height_1400_bw/walmart_2.png')
  
# Convert image into grayscale image
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  
# Setting binary threshold of gray image
_, threshold = cv2.threshold(gray, 254, 255, cv2.THRESH_BINARY)
  
# Use a findContours() function
contours, _ = cv2.findContours(
    threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

with_contours = cv2.drawContours(img, contours, -1, (255,0,255), 3)
  
i = 0
  
# List for storing names of shapes
for contour in contours:
  
    # Ignore first counter bc findContours() function detects whole image as shape
    if i == 0:
        i = 1
        continue
  
    # cv2.approxPloyDP() function to approximate the shape
    approx = cv2.approxPolyDP(
        contour, 0.01 * cv2.arcLength(contour, True), True)
      
    # Use drawContours() function
    cv2.drawContours(img, [contour], 0, (0, 0, 255), 5)
  
    # Find center point of shape (x_center, y_center)
    M = cv2.moments(contour)
    if M['m00'] != 0.0:
        x_center = int(M['m10']/M['m00'])
        y_center = int(M['m01']/M['m00'])
        
        
    # Draw a bounding box around all contours
    x, y, w, h = cv2.boundingRect(contour)
    #with_contours = cv2.drawContours(img, contours, -1, (255,0,255), 3)
 
    # Make sure contour area is large enough
    #if (cv2.contourArea(contour)) > 10:
    cv2.rectangle(with_contours, (x,y), (x+w,y+h), (255,0,0), 5)
        
        
    # Make a square image with the contour of the shape at boundaries of the square
    # https://automaticaddison.com/how-to-detect-and-draw-contours-in-images-using-opencv/
  
    # putting shape name at center of each shape
    if len(approx) == 3:
        cv2.putText(img, 'Triangle', (x, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
  
    elif len(approx) == 4:
        cv2.putText(img, 'Quadrilateral', (x, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
  
    elif len(approx) == 5:
        cv2.putText(img, 'Pentagon', (x, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
  
    elif len(approx) == 6:
        cv2.putText(img, 'Hexagon', (x, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
  
    else:
        cv2.putText(img, 'circle', (x_center, y_center),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
  
# displaying the image after drawing contours
cv2.imwrite('shape_test.png', img)

True

In [67]:
def contour_img(img_path, company_name, write_dir):
    # Project: How to Detect and Draw Contours in Images Using OpenCV (2)
    
    # Read the color image
    image = cv2.imread(img_path)

    new_image   = image.copy() # Make a copy
    gray        = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Convert the image to grayscale
    ret, binary = cv2.threshold(gray, 100, 255, cv2.THRESH_OTSU) # Convert the grayscale image to binary

    inverted_binary = ~binary # black on white --> white on black

    # Find the contours on the inverted binary image, and store them in a list
    # Contours are drawn around white blobs. 
    # Hierarchy variable contains info on the relationship between the contours
    contours, hierarchy = cv2.findContours(inverted_binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw the contours (in red) on the original image and display the result
    # Input color code is in BGR (blue, green, red) format
    # -1 means to draw all contours
    with_contours = cv2.drawContours(image, contours, -1, (255,0,255), 3)

    # Show the total number of contours that were detected
    print('Total number of contours detected: ' + str(len(contours)))

    # Draw just the first contour
    # The 0 means to draw the first contour
    first_contour = cv2.drawContours(new_image, contours, 0, (255,0,255), 3)

    try: 
        # Draw a bounding box around the first contour
        # x:  starting x coordinate of the bounding box
        # y:  starting y coordinate of the bounding box
        # w:  width of the bounding box
        # h:  height of the bounding box
        x, y, w, h = cv2.boundingRect(contours[0])
        cv2.rectangle(first_contour,(x,y), (x+w,y+h), (255,0,0), 5)

        contour_num = 0
        shapes      = []
        # Draw a bounding box around all contours
        for c in contours:
            x, y, w, h    = cv2.boundingRect(c)
            contour_num  += 1
            
            # Mask out all other contours in the image
            mask = np.ones(inverted_binary.shape[:2], dtype="uint8") * 255
            for other_c in contours: 
                # if the contour is not the current one, draw it on the mask
                if other_c is not c:
                    cv2.drawContours(mask, [other_c], -1, 0, -1)
                    
            masked_img = cv2.bitwise_and(inverted_binary, inverted_binary, mask=mask)
            
            #contour = np.zeros(inverted_binary.shape[:2], dtype = "uint8")
            ctr = np.array(c).reshape((-1,1,2)).astype(np.int32)
            cornerMask = np.zeros(image.shape[:2], dtype = "uint8")
            cv2.rectangle(cornerMask, (x, y), (x+w, y+h), 255, -1) 
            cv2.drawContours(cornerMask,[ctr],0,(0,0,0),-1)
         
            # See if center point of shape is black

            # Make sure contour area is large enough
            if (cv2.contourArea(c)) > 10:
                cv2.rectangle(with_contours,(x,y), (x+w,y+h), (255,0,0), 5)

                # See if center point of shape is black

                # Make different images for each contour
                shape = masked_img[y:y+h,x:x+w] # black background, gets rid of inside contours?
                #shape = cornerMask[y:y+h,x:x+w] # white background, just one contour
                shapes.append(shape)
                #cv2.imwrite("contour_" + str(contour_num) + ".png", shape)

                # Delete a contour if in the bounding box but not inside this current contour
                # https://stackoverflow.com/questions/13786088/determine-if-a-point-is-inside-or-outside-of-a-shape-with-opencv/13786159#:~:text=To%20determine%20if%20a%20point,or%20on%20the%20contour%2C%20respectively.

                # Add black outer padding to the width or height to make a square figure
                dimension_diff = abs(w-h)
                padding_1 = math.ceil(dimension_diff/2)
                padding_2 = math.floor(dimension_diff/2)
                PAD = 20
                if w > h: 
                    # top, bottom, left, right
                    shape_square = cv2.copyMakeBorder(shape,
                                                      PAD + padding_1,
                                                      PAD + padding_2,
                                                      PAD,
                                                      PAD,
                                                      cv2.BORDER_CONSTANT,
                                                      value = BLACK)
                else: 
                    shape_square = cv2.copyMakeBorder(shape,
                                                      PAD,
                                                      PAD,
                                                      PAD + padding_1,
                                                      PAD + padding_2,
                                                      cv2.BORDER_CONSTANT,
                                                      value = BLACK)

                #shape_square = cv2.copyMakeBorder(shape, 10, 10, 10, 10, cv2.BORDER_CONSTANT, value=BLACK)
                
                cv2.imwrite(write_dir + "/contour_square_" + 
                            company_name[:-4] + "_" + 
                            str(contour_num) + ".png", shape_square)

#         cv2.imwrite("gray.png", gray) 
#         cv2.imwrite("binary.png", binary)
#         cv2.imwrite('inverted_binary.png', inverted_binary)
#         cv2.imwrite('detected_contours.png', with_contours)
#         cv2.imwrite('first_contour.png', first_contour)
#         cv2.imwrite('first_bounding_box.png', first_contour)
#         cv2.imwrite('boundary_box_test.png', with_contours)
#         cv2.imwrite("MASK_TEST.png", masked_img)
        
    except Exception as e: 
        print("Error:" + str(e))


In [61]:
contour_img("full_png_height_1400_bw/walmart_2.png", "walmart_2.png", "")

Total number of contours detected: 19


In [68]:
# Convert contours to black, full-size squares
read_dir = "full_png_height_1400_bw"
write_dir = "full_png_black_contour_squares"
 
# Iterate over files in that directory
for filename in os.listdir(read_dir):
    img = os.path.join(read_dir, filename)
    # checking if it is a file
    if os.path.isfile(img):
        contour_img(img, filename, write_dir)

Total number of contours detected: 17
Total number of contours detected: 50
Total number of contours detected: 13
Total number of contours detected: 16
Total number of contours detected: 16
Total number of contours detected: 13
Total number of contours detected: 25
Total number of contours detected: 7
Total number of contours detected: 13
Total number of contours detected: 4
Total number of contours detected: 13
Total number of contours detected: 12
Total number of contours detected: 39
Total number of contours detected: 3
Total number of contours detected: 25
Total number of contours detected: 77
Total number of contours detected: 9
Total number of contours detected: 21
Total number of contours detected: 29
Total number of contours detected: 21
Total number of contours detected: 11
Total number of contours detected: 14
Total number of contours detected: 7
Total number of contours detected: 11
Total number of contours detected: 0
Error:tuple index out of range
Total number of contours 

Total number of contours detected: 12
Total number of contours detected: 23
Total number of contours detected: 5
Total number of contours detected: 26
Total number of contours detected: 14
Total number of contours detected: 22
Total number of contours detected: 23
Total number of contours detected: 16
Total number of contours detected: 33
Total number of contours detected: 34
Total number of contours detected: 20
Total number of contours detected: 18
Total number of contours detected: 23
Total number of contours detected: 21
Total number of contours detected: 26
Total number of contours detected: 10
Total number of contours detected: 16
Total number of contours detected: 9
Total number of contours detected: 11
Total number of contours detected: 30
Total number of contours detected: 26
Total number of contours detected: 27
Total number of contours detected: 27
Total number of contours detected: 13
Total number of contours detected: 7
Total number of contours detected: 12
Total number of

Total number of contours detected: 28
Total number of contours detected: 18
Total number of contours detected: 25
Total number of contours detected: 32
Total number of contours detected: 6
Total number of contours detected: 6
Total number of contours detected: 27
Total number of contours detected: 53
Total number of contours detected: 9
Total number of contours detected: 1
Total number of contours detected: 23
Total number of contours detected: 9
Total number of contours detected: 56
Total number of contours detected: 9
Total number of contours detected: 11
Total number of contours detected: 14
Total number of contours detected: 13
Total number of contours detected: 9
Total number of contours detected: 0
Error:tuple index out of range
Total number of contours detected: 25
Total number of contours detected: 21
Total number of contours detected: 20
Total number of contours detected: 30
Total number of contours detected: 22
Total number of contours detected: 27
Total number of contours de

In [71]:
# Convert to 28x28 pixel samples
read_dir = "full_png_black_contour_squares"
write_dir = "28_png_black_contour_squares"
 
# Iterate over files in that directory
for filename in os.listdir(read_dir):
    img = os.path.join(read_dir, filename)
    # checking if it is a file
    if os.path.isfile(img):
        big_img = cv2.imread(img, 0)
        if big_img is not None and big_img.shape[0] > 0 and big_img.shape[1] > 0:
            if cv2.countNonZero(big_img) > 0:
                dim = (28, 28)
                resized = cv2.resize(big_img, dim, interpolation = cv2.INTER_AREA)
                cv2.imwrite(write_dir + "/" + str(big_img.shape[0]) + "_" + filename[:-4] + ".png", resized)


In [13]:
# Convert png to two-color (just black and just white image)

file = "full_png_height_1400/walmart_2.png"
img = Image.open(file)
img.show()

img = img.convert("1")
img.show()

In [ ]:
    
    # read svg file -> write png file
    # cairosvg.svg2png(url=input_svg_path, write_to=output_png_path, output_width=width, output_height=height)

    # read svg file -> png data
    # png_data = cairosvg.svg2png(url=input_svg_path, output_width=width, output_height=height)

    # svg string -> write png file
    # cairosvg.svg2png(bytestring=svg_str.encode(), write_to=output_png_path, output_width=width, output_height=height)

    # svg string -> png data
    # png_data = cairosvg.svg2png(bytestring=svg_str.encode(), output_width=width, output_height=height)
    
    
    
    
    
    
#    response = requests.get(url)
#    img      = Image.open(BytesIO(response.content))

#    img_normal, img_rotated, img_flipped = "normal.jpg", "rotated.jpg", "flipped.jpg"
#    im_enhancer = ImageEnhance.Contrast(img)

#    contrast_factor = 2  #increase contrast in original image
#    img = im_enhancer.enhance(contrast_factor)
#    img.save("test_" + img_num + ".png")

#     im_normal, im_rotated, im_flipped = "normal.jpg", "rotated.jpg", "flipped.jpg"

#     directory_name = "attempts/brooklyn_rail/attempt_" + im_num + "_letters_" + im_name
#     if not os.path.exists(directory_name):
#         os.makedirs(directory_name)

#     best_responses = [dict() for x in range(nb_classes)]

#     index = 0
#     ratio = np.power(2, index)
#     sample_len = 28 * ratio

#     step  = 4 * (index + 1)
#     width, height = img.size[0], img.size[1]

#     while sample_len < np.minimum(width,height):
#     # while ratio < 4:
#         print(sample_len)

#         start_i = width  - sample_len
#         start_j = height - sample_len

#         step = step * ratio
#         i_passes = int(np.floor(start_i/step))
#         j_passes = int(np.floor(start_j/step))

#         for i in range( i_passes ): 
#             for j in range( j_passes ):

#                 for angle in [0,90,180,270]:

#                     step_i, step_j = (step * i), (step * j)

#                     im_cropped = img.crop( (step_i, step_j, step_i+sample_len, step_j+sample_len) )
#                     im_cropped.save(im_normal)
#                     im_cropped.rotate(angle).save(im_rotated)
#                     im_cropped.rotate(angle).transpose(Image.FLIP_LEFT_RIGHT).save(im_flipped)

#                     x     = imread(im_normal,  mode='L')
#                     x_rot = imread(im_rotated, mode='L')
#                     x_flp = imread(im_flipped, mode='L')

#                     x_rot = imresize(x_rot,(28,28))
#                     x_flp = imresize(x_flp,(28,28))

#                     best_responses = predict(x, x_rot, best_responses, step_i, step_j, angle)   # test normal image, rotated
#                     best_responses = predict(x, x_flp, best_responses, step_i, step_j, angle)   # test normal image, rotated & mirrored

#                     x_rot = np.invert(x_rot)
#                     x_flp = np.invert(x_flp)

#                     best_responses = predict(x, x_rot, best_responses, step_i, step_j, angle)   # test inverted image, rotated
#                     best_responses = predict(x, x_flp, best_responses, step_i, step_j, angle)   # test inverted image, rotated & mirrored

#         index += 1
#         ratio = np.power(2, index)
#         sample_len = 28 * ratio

#     elapsed = timeit.default_timer() - start_time
#     print(elapsed)

#     #----------------------------------------------------------------------

#     im = np.array(img, dtype=np.uint8)

#     # Create figure and axes
#     fig,ax = plt.subplots(1)

#     # Display the image
#     ax.imshow(im)

#     results_directory_name = "results/brooklyn_rail"
#     if not os.path.exists(results_directory_name):
#         os.makedirs(results_directory_name)

#     results_file_name = im_num + "_" + im_name
#     info_name         = results_directory_name + results_file_name + ".txt"
#     fig_name          = results_directory_name + results_file_name + ".png"

#     with open(info_name, 'w') as f:
#         f.write(im_name + "\n")
#         f.write(im_num + "_small" + "\n")

#         for i in range (nb_classes): 
#             if not best_responses[i]:
#                 f.write("EMPTY\n")
#             else:
#                 x_ = best_responses[i]['image']

#                 response = {'prediction': best_responses[i]['prediction'],
#                             'confidence': best_responses[i]['confidence'],
#                             'sample_len': best_responses[i]['image'].shape,
#                             'box': best_responses[i]['box']}
#                 f.write(str(response) + "\n")

#                 # Create a Rectangle patch
#                 box_len  = best_responses[i]['image'].shape[0]
#                 top_left = best_responses[i]['box']
#                 bot_left = (top_left[0], top_left[1])

#                 rect = patches.Rectangle( bot_left, box_len, box_len,
#                                           linewidth=1, edgecolor='r',
#                                           facecolor='none' )

#                 # Add the patch to the plot
#                 ax.add_patch(rect)

#                 im = Image.fromarray(best_responses[i]['image'])
#                 im.save(directory_name + "/" + best_responses[i]['prediction'] + ".jpg")

#                 im_orig = Image.fromarray(best_responses[i]['image_init'])
#                 im_orig.save(directory_name + "/_" + best_responses[i]['prediction'] + ".jpg")

#             f.write("\n")

#         plt.savefig(fig_name, dpi=1000)

#     print("Done!")